# Accumulation-Spending Gap Optimization for Game Economy

### Setup

As we have to generate the optimized set of integer values while fulfilling different non-complex conditions, **Linear Programming with constraints** would be a good fit. This can be done through **pulp** (python's linear programming library). 

The variables needed to be optimized in this:

1. The Accumulation Variables: 
    1. Easy (E)
    2. Medium (M)
    3. Hard (H)
<br>
2. The Spending Variables:
    1. Numbers (N)
    2. Operations (O)
    3. Specials (S)

Lower Bound for each variables is set at 1 and appropriate Upper Bounds were set to avoid energy counts becoming overly large numbers during the game.


In [3]:
from pulp import *
import pandas as pd

# Defining the problem and variables
prob = LpProblem("MathscryptionBalance", LpMinimize)

E = LpVariable("E", lowBound=1, upBound=3, cat='Integer')   #Easy Question
M = LpVariable("M", lowBound=1, upBound=4, cat='Integer')   #Med Question
H = LpVariable("H", lowBound=1, upBound=5, cat='Integer')   #Hard Question

N = LpVariable("N", lowBound=1, upBound=3, cat='Integer')   #Numbered Cards
O = LpVariable("O", lowBound=1, upBound=4, cat='Integer')   #Operation Notes
S = LpVariable("S", lowBound=1, upBound=5, cat='Integer')   #Special Cards

### ModelOne: Realistic Only

We start with a relatively simple model by hypothesizing the realistic number of questions answered of each type and the realistic number of card bought of each type to set up equations for accumulation and spending.
Additionally, we set up the equations for maximum possible accumualtion and spending.

We want to ensure that:
1. The A-S gap for realistic values is minimized.
2. The maximum possible accumulation is greater than maximum possible spending so the best performers can play with the most freedom, while also carrying extra energy to the next round.

Other economy constraints inlcude:
1. Spending for an Operation cards should be more than or equal to a Numbered card.
2. Accumulation for Easy, Medium and Hard Questions should be three consecutive integers.
3. Spending on a Spedcial card should be atleast 2 more energy than an operaton card.

In [4]:
from pulp import *

# Defining the problem and variables
prob = LpProblem("MathscryptionBalance", LpMinimize)

E = LpVariable("E", lowBound=1, upBound=3, cat='Integer')   #Easy Question
M = LpVariable("M", lowBound=1, upBound=4, cat='Integer')   #Med Question
H = LpVariable("H", lowBound=1, upBound=5, cat='Integer')   #Hard Question

N = LpVariable("N", lowBound=1, upBound=3, cat='Integer')   #Numbered Cards
O = LpVariable("O", lowBound=1, upBound=4, cat='Integer')   #Operation Notes
S = LpVariable("S", lowBound=1, upBound=5, cat='Integer')   #Special Cards

# Realistic and max values

realistic_energy = 5*E + 15*M + 5*H
realistic_spend = 15*N + 10*O + 5*S

max_energy = 25*H
max_spend = 20*N + 15*O + 6*S

# Introducing a variable to model the absolute gap
REALGAP = LpVariable("REALGAP", lowBound=0, cat='Continuous')
MAXGAP = LpVariable("MAXGAP", lowBound=0, cat='Continuous')

# Setting constraints for absolute value
prob += REALGAP >= realistic_energy - realistic_spend
prob += REALGAP >= realistic_spend - realistic_energy


# Additional constraint
prob += O >= N
prob += M == E+1
prob += H == M+1
prob += S == O+2

#Ensuring max energy is more than maximum possible spend
prob += max_energy >= max_spend

prob += REALGAP

# Solve
prob.solve()

# Output
print("Status:", LpStatus[prob.status])

print(f"E (Easy energy) = {E.varValue}")
print(f"M (Medium energy) = {M.varValue}")
print(f"H (Hard energy) = {H.varValue}")
print(f"N (Number card cost) = {N.varValue}")
print(f"O (Operation card cost) = {O.varValue}")
print(f"S (Special card cost) = {S.varValue}")

print(f"Realistic Energy = {value(realistic_energy)}")
print(f"Realistic Spend = {value(realistic_spend)}")
print(f"Max Energy = {value(max_energy)}")
print(f"Max Spend = {value(max_spend)}")

print(f"Optimized Realistic GAP = {value(REALGAP)}")


Status: Optimal
E (Easy energy) = 2.0
M (Medium energy) = 3.0
H (Hard energy) = 4.0
N (Number card cost) = 1.0
O (Operation card cost) = 3.0
S (Special card cost) = 5.0
Realistic Energy = 75.0
Realistic Spend = 70.0
Max Energy = 100.0
Max Spend = 95.0
Optimized Realistic GAP = 5.0


### ModelTwo: Maximum and Realistic

The tweaks in the second model from the first are as follows:
1. We inroduce a variable for the A-S gap between the Max values as well.
2. This variable needs to be minimized as well.
3. The optimized solution considers the minimization of both A-S Gaps - Realistic and Maximum.
4. Displays the average optimized value of Realsitic and Maximum A-S Gap.

In [5]:
from pulp import *

# Defining the problem and variables
prob = LpProblem("MathscryptionBalance", LpMinimize)

E = LpVariable("E", lowBound=1, upBound=3, cat='Integer')   #Easy Question
M = LpVariable("M", lowBound=2, upBound=4, cat='Integer')   #Med Question
H = LpVariable("H", lowBound=3, upBound=5, cat='Integer')   #Hard Question

N = LpVariable("N", lowBound=1, upBound=3, cat='Integer')   #Numbered Cards
O = LpVariable("O", lowBound=1, upBound=4, cat='Integer')   #Operation Notes
S = LpVariable("S", lowBound=1, upBound=5, cat='Integer')   #Special Cards

realistic_energy = 5*E + 15*M + 5*H
realistic_spend = 15*N + 10*O + 5*S

max_energy = 25*H
max_spend = 20*N + 15*O + 6*S

# Introducing a variable to model the absolute gap
REALGAP = LpVariable("REALGAP", lowBound=0, cat='Continuous')
MAXGAP = LpVariable("MAXGAP", lowBound=0, cat='Continuous')

# Setting constraints for absolute value
prob += REALGAP >= realistic_energy - realistic_spend
prob += REALGAP >= realistic_spend - realistic_energy

prob += MAXGAP >= max_energy - max_spend
prob += MAXGAP >= max_spend - max_energy

# Additional constraint
prob += O >= N
prob += M == E+1
prob += H == M+1
prob += S == O+2

#Ensuring max energy is more than maximum possible spend
prob += max_energy >= max_spend

# Minimizing both the maximum case gap and the realistic case gap
prob += MAXGAP+REALGAP

# Solve
prob.solve()

# Output
print("Status:", LpStatus[prob.status])

print(f"E (Easy energy) = {E.varValue}")
print(f"M (Medium energy) = {M.varValue}")
print(f"H (Hard energy) = {H.varValue}")
print(f"N (Number card cost) = {N.varValue}")
print(f"O (Operation card cost) = {O.varValue}")
print(f"S (Special card cost) = {S.varValue}")

print(f"Realistic Energy = {value(realistic_energy)}")
print(f"Realistic Spend = {value(realistic_spend)}")
print(f"Max Energy = {value(max_energy)}")
print(f"Max Spend = {value(max_spend)}")

print(f"Realistic GAP = {value(REALGAP)}")
print(f"Max GAP = {value(MAXGAP)}")

print(f"Average Optimized GAP = {value((MAXGAP+REALGAP)/2)}")


Status: Optimal
E (Easy energy) = 1.0
M (Medium energy) = 2.0
H (Hard energy) = 3.0
N (Number card cost) = 1.0
O (Operation card cost) = 2.0
S (Special card cost) = 4.0
Realistic Energy = 50.0
Realistic Spend = 55.0
Max Energy = 75.0
Max Spend = 74.0
Realistic GAP = 5.0
Max GAP = 1.0
Average Optimized GAP = 3.0


### ModelThree: Maximum and Levels of Realistic

Different participants will have different levels of performances in the accumulation stage, which affects their spending strategy accordingly. Therefore, we design the programs to optimize A-S gaps across different levels of realistic:
1. Set up 3 different levels of realsitic performances and defined their accumulation and spending equations
2. Defined their gaps as variables that need to be minimized.
3. Optimize all 4 gaps - Real1, Real2, Real3, Max.
4. Display average optimized value for all 4 A-S gaps.

In [6]:
from pulp import *

# Defining the problem and variables
prob = LpProblem("MathscryptionBalance", LpMinimize)

E = LpVariable("E", lowBound=1, upBound=3, cat='Integer')   #Easy Question
M = LpVariable("M", lowBound=1, upBound=4, cat='Integer')   #Med Question
H = LpVariable("H", lowBound=1, upBound=5, cat='Integer')   #Hard Question

N = LpVariable("N", lowBound=1, upBound=3, cat='Integer')   #Numbered Cards
O = LpVariable("O", lowBound=1, upBound=4, cat='Integer')   #Operation Notes
S = LpVariable("S", lowBound=1, upBound=5, cat='Integer')   #Special Cards

#Different player levels and extreme maximum value scenarios

Lev1_energy = 5*E + 15*M
Lev1_spend = 15*N + 10*O + 3*S

Lev2_energy = 5*E + 15*M + 5*H
Lev2_spend = 16*N + 11*O + 4*S

Lev3_energy = 15*M + 10*H
Lev3_spend = 18*N + 13*O + 5*S

Max_energy = 25*H
Max_spend = 20*N + 15*O + 6*S

# Introducing variables for accumultion-spending gaps for each level

LEV1GAP = LpVariable("LEV1GAP", lowBound=0, cat='Continuous')
LEV2GAP = LpVariable("LEV2GAP", lowBound=0, cat='Continuous')
LEV3GAP = LpVariable("LEV3GAP", lowBound=0, cat='Continuous')
MAXGAP = LpVariable("MAXGAP", lowBound=0, cat='Continuous')

# Setting constraints for absolute value

prob += LEV1GAP >= Lev1_energy - Lev1_spend
prob += LEV1GAP >= Lev1_spend - Lev1_energy

prob += LEV2GAP >= Lev2_energy - Lev2_spend
prob += LEV2GAP >= Lev2_spend - Lev2_energy

prob += LEV3GAP >= Lev3_energy - Lev3_spend
prob += LEV3GAP >= Lev3_spend - Lev3_energy

prob += MAXGAP >= Max_energy - Max_spend
prob += MAXGAP >= Max_spend - Max_energy

#Question Energies constraints
prob += M == E+1
prob += H == M+1

#Card Prices constraints
prob += O <= N+1
prob += S == O+2

#Ensuring max energy is more than maximum possible spend
prob += Max_energy >= Max_spend


# Main Objective: Accumulation-Spending Gap Minimization
prob += LEV1GAP+LEV2GAP+LEV3GAP+MAXGAP

# Solve
prob.solve()

# Output
print("Status:", LpStatus[prob.status])
print(f"E (Easy energy) = {E.varValue}")
print(f"M (Medium energy) = {M.varValue}")
print(f"H (Hard energy) = {H.varValue}")
print(f"N (Number card cost) = {N.varValue}")
print(f"O (Operation card cost) = {O.varValue}")
print(f"S (Special card cost) = {S.varValue}")

print(f"LEVEL1 Energy = {value(Lev1_energy)}")
print(f"LEVEL1 Spend = {value(Lev1_spend)}")
print(f"LEVEL2 Energy = {value(Lev2_energy)}")
print(f"LEVEL2 Spend = {value(Lev2_spend)}")
print(f"LEVEL3 Energy = {value(Lev3_energy)}")
print(f"LEVEL3 Spend = {value(Lev3_spend)}")
print(f"Max Energy = {value(Max_energy)}")
print(f"Max Spend = {value(Max_spend)}")

print(f"LEVEL1 GAP = {value(LEV1GAP)}")
print(f"LEVEL2 GAP = {value(LEV2GAP)}")
print(f"LEVEL3 GAP = {value(LEV3GAP)}")
print(f"MAX GAP = {value(MAXGAP)}")

print(f"Optimized Average GAP = {value((LEV1GAP+LEV2GAP+LEV3GAP+MAXGAP)/4)}")


Status: Optimal
E (Easy energy) = 2.0
M (Medium energy) = 3.0
H (Hard energy) = 4.0
N (Number card cost) = 2.0
O (Operation card cost) = 2.0
S (Special card cost) = 4.0
LEVEL1 Energy = 55.0
LEVEL1 Spend = 62.0
LEVEL2 Energy = 75.0
LEVEL2 Spend = 70.0
LEVEL3 Energy = 85.0
LEVEL3 Spend = 82.0
Max Energy = 100.0
Max Spend = 94.0
LEVEL1 GAP = 7.0
LEVEL2 GAP = 5.0
LEVEL3 GAP = 3.0
MAX GAP = 6.0
Optimized Average GAP = 5.25


### Conclusion

Average Optimized Gaps for the three models were as follows:
1. One: 5.00
2. Two: 3.00
3. Three: 5.25  

Despite Two having the best ptimization score, option Three was the most accurate model for the game, taking the different levels of expected play into account and optimizing across them. 
Hence, option 3 was determined to be the best model and Accumulation-Spending figures were set as follows:
1. Easy Questions gives 2 Energy
1. Medium Question gives 3 Energy
1. Hard Question gives 4 Energy
1. Number Cards costs 2 Energy
1. Operation Cards costs 2 Energy
1. Special Cards costs 4 Energy